In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import gc
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans as SklearnKMeans

def select_keyframes(video_path, clusters=3, num_keyframes=3):
    reader = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    if not reader.isOpened():
        print(f"Failed to open video file: {video_path}")
        return []

    while True:
        ret, frame = reader.read()
        if not ret:
            break
        if frame_count % 30 == 0:  # Select every 30th frame
            frames.append(frame)
        frame_count += 1
    reader.release()

    if len(frames) == 0:
        print(f"No frames read from the video file: {video_path}")
        return []

    frames_downsized = [cv2.resize(frame, (frame.shape[1] // 4, frame.shape[0] // 4)) for frame in frames]
    frames_downsized = np.array(frames_downsized)
    frames_downsized_flattened = frames_downsized.reshape(frames_downsized.shape[0], -1)

    pca = PCA(n_components=2)
    frames_2dim = pca.fit_transform(frames_downsized_flattened)

    kmeans = SklearnKMeans(n_clusters=clusters, random_state=0, n_init=10)
    kmeans.fit(frames_2dim)

    labels = kmeans.labels_
    centroid_positions = kmeans.cluster_centers_

    frame_scores = np.ones(len(frames))  # Initialize all scores to 1 (bad keyframes)

    for i in range(clusters):
        input_data = frames_2dim[labels == i]
        input_indices = np.argwhere(labels == i)
        closest_index = np.argmin(np.abs(input_data - centroid_positions[i]).sum(axis=1))
        abs_index = input_indices[closest_index]
        frame_scores[abs_index[0]] = 5  # Assign highest score to keyframes selected by our method

    return frame_scores

# Step 1: Load Data
base_path = '/AutogenerateVideoDescriptions/TVSum'
info_path = os.path.join(base_path, 'ydata-tvsum50-data/data/ydata-tvsum50-info.tsv')
anno_path = os.path.join(base_path, 'ydata-tvsum50-data/data/ydata-tvsum50-anno.tsv')
videos_path = os.path.join(base_path, 'ydata-tvsum50-video/video')

info_df = pd.read_csv(info_path, sep='\t', header=None)
anno_df = pd.read_csv(anno_path, sep='\t', header=None)

info_df.columns = ['category', 'video_id', 'title', 'url', 'length']
anno_df.columns = ['video_id', 'category', 'scores']

anno_df['scores'] = anno_df['scores'].apply(lambda x: list(map(int, x.strip('[]').split(','))))

# Step 2: Process all videos and save scores
all_scores = []

for video_id in info_df['video_id']:
    print(f"Processing video ID {video_id}...")
    video_path = os.path.join(videos_path, f'{video_id}.mp4')
    if not os.path.exists(video_path):
        print(f"Video file does not exist: {video_path}")
        continue

    frame_scores = select_keyframes(video_path, clusters=5, num_keyframes=6)

    if len(frame_scores) == 0:
        print(f"No frames scored for video ID {video_id}.")
        continue

    # Assign scores to every 30th frame based on keyframe selection
    video_scores = []
    total_frames = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT))

    for i in range(total_frames):
        if i % 30 == 0:
            frame_index = i // 30
            if frame_index < len(frame_scores):
                video_scores.append(frame_scores[frame_index])
            else:
                video_scores.append(1)  # Default to 1 if frame_index is out of range
        else:
            video_scores.append(np.random.randint(1, 6))  # Random score between 1 and 5

    all_scores.append({'video_id': video_id, 'scores': video_scores})
    print(f"Assigned frame scores for video ID {video_id}")

    # Free up memory
    del frame_scores
    gc.collect()

# Convert to DataFrame and save to CSV
scores_df = pd.DataFrame(all_scores)
scores_df.to_csv('video_frame_scores.csv', index=False)
print("Scores have been saved to video_frame_scores.csv")


In [ ]:
scores_df

,video_id,scores
0,AwmHb44_ouw,"[1.0, 3, 1, 4, 2, 3, 2, 3, 1, 2, 3, 5, 5, 1, 3..."
1,98MoyGZKHXc,"[1.0, 2, 2, 4, 2, 3, 1, 5, 5, 3, 5, 1, 3, 5, 4..."
2,J0nA4VgnoCo,"[1.0, 4, 2, 4, 2, 3, 2, 1, 3, 5, 1, 5, 1, 5, 3..."
3,gzDbaEs1Rlg,"[1.0, 4, 1, 5, 5, 5, 2, 1, 2, 1, 1, 1, 4, 1, 2..."
4,XzYM3PfTM4w,"[1.0, 2, 4, 3, 3, 5, 5, 3, 3, 5, 1, 3, 5, 2, 3..."
5,HT5vyqe0Xaw,"[1.0, 1, 5, 2, 1, 5, 3, 4, 2, 4, 4, 3, 3, 5, 2..."
6,sTEELN-vY30,"[1.0, 1, 5, 3, 4, 1, 1, 3, 2, 4, 3, 1, 4, 5, 2..."
7,vdmoEJ5YbrQ,"[1.0, 5, 3, 4, 5, 4, 5, 1, 4, 4, 4, 3, 2, 1, 1..."
8,xwqBXPGE9pQ,"[1.0, 2, 3, 1, 5, 1, 1, 2, 5, 2, 3, 3, 5, 3, 1..."
9,akI8YFjEmUw,"[1.0, 3, 4, 4, 4, 4, 2, 2, 1, 2, 1, 2, 4, 4, 3..."


In [ ]:
# Load video frame scores from the CSV file
scores_df = pd.read_csv('video_frame_scores.csv')

# Define keyframe_scores_dict from loaded DataFrame
keyframe_scores_dict = {}
for video_id, scores in zip(scores_df['video_id'], scores_df['scores']):
    keyframe_scores_dict[video_id] = eval(scores)

# Load annotations DataFrame skipping the first row
anno_df = pd.read_csv(anno_path, sep='\t', header=None, skiprows=1)
anno_df.columns = ['video_id', 'category', 'scores']
anno_df['scores'] = anno_df['scores'].apply(lambda x: list(map(int, x.strip('[]').split(','))))



In [ ]:
type(anno_df['scores'][0])

list

In [ ]:
# Calculate average scores for frames
average_frame_scores = []

for video_id, scores in keyframe_scores_dict.items():
    # Get scores for the current video from annotations DataFrame
    video_annotations = anno_df[anno_df['video_id'] == video_id]['scores'].tolist()

    # Check if annotation scores are empty
    if len(video_annotations) == 0:
        print(f"No annotation scores found for video ID {video_id}. Skipping...")
        continue

    # Transpose annotation scores to align with keyframe scores
    video_annotations_transpose = np.array(video_annotations).T

    # Calculate average score for each frame across all annotators
    avg_frame_scores = np.mean(video_annotations_transpose, axis=1)

    # Add average scores to the list
    average_frame_scores.append({'video_id': video_id, 'average_scores': avg_frame_scores})

# Convert to DataFrame
average_frame_scores_df = pd.DataFrame(average_frame_scores)

# Save to CSV
average_frame_scores_df.to_csv('average_scores.csv', index=False)
print("Average frame scores have been saved to average_frame_scores.csv")


Average frame scores have been saved to average_frame_scores.csv


In [ ]:
scores_df['scores'][0]

'[1.0, 3, 1, 4, 2, 3, 2, 3, 1, 2, 3, 5, 5, 1, 3, 1, 1, 1, 1, 3, 2, 2, 5, 3, 2, 2, 5, 2, 2, 1, 1.0, 5, 1, 5, 4, 1, 4, 2, 1, 4, 4, 1, 5, 1, 1, 2, 3, 1, 2, 4, 4, 5, 4, 5, 4, 2, 4, 4, 1, 4, 1.0, 1, 2, 4, 2, 2, 3, 5, 1, 4, 3, 5, 5, 2, 1, 3, 4, 5, 3, 2, 1, 5, 2, 3, 4, 2, 1, 4, 2, 2, 1.0, 5, 2, 5, 5, 4, 2, 3, 2, 4, 2, 3, 5, 1, 1, 1, 3, 5, 2, 3, 1, 5, 1, 1, 5, 2, 1, 2, 5, 4, 1.0, 3, 5, 2, 1, 2, 3, 5, 4, 5, 4, 3, 1, 2, 4, 5, 2, 1, 1, 4, 3, 4, 4, 3, 4, 5, 2, 1, 1, 4, 1.0, 3, 1, 3, 3, 4, 4, 3, 1, 1, 3, 2, 1, 5, 4, 5, 4, 5, 2, 4, 2, 2, 3, 4, 4, 1, 2, 1, 4, 4, 1.0, 2, 4, 2, 2, 4, 2, 1, 2, 4, 3, 1, 1, 4, 3, 3, 3, 1, 4, 3, 2, 2, 5, 1, 1, 1, 3, 5, 5, 3, 1.0, 3, 1, 5, 5, 3, 4, 4, 5, 1, 5, 1, 3, 1, 2, 4, 5, 4, 1, 2, 5, 4, 5, 1, 2, 1, 3, 4, 2, 2, 1.0, 2, 5, 4, 1, 5, 5, 1, 4, 5, 4, 5, 1, 2, 1, 5, 1, 4, 4, 5, 2, 3, 3, 2, 3, 2, 1, 5, 3, 2, 1.0, 3, 3, 2, 2, 4, 4, 1, 2, 2, 2, 1, 3, 3, 4, 4, 1, 4, 3, 5, 3, 3, 1, 3, 4, 5, 2, 1, 4, 4, 1.0, 2, 3, 1, 1, 1, 4, 1, 1, 4, 1, 3, 5, 1, 4, 3, 2, 1, 5, 4, 5, 2, 5, 4, 2, 3

In [ ]:
total_last_df = average_frame_scores_df
total_last_df['my_scores'] = scores_df['scores']

In [ ]:
type(total_last_df['my_scores'][0])

str

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, average_precision_score


column1 = total_last_df['average_scores'].apply(np.array)
column2 = total_last_df['my_scores'].apply(np.array)
for i in range(len(column2)):
  column2[i] = column2[i].replace('1.0','1')
  column2[i] = column2[i].replace('2.0','2')
  column2[i] = column2[i].replace('3.0','3')
  column2[i] = column2[i].replace('4.0','4')
  column2[i] = column2[i].replace('5.0','5')

# column2[i].apply(lambda x: list(map(int, x.strip('[]').split(',')))) -> поменять и превратить в np.array.

In [ ]:
column2[i]

'[1, 4, 1, 5, 5, 5, 2, 1, 2, 1, 1, 1, 4, 1, 2, 5, 4, 2, 3, 3, 5, 1, 4, 4, 3, 3, 5, 3, 1, 2, 1, 3, 4, 1, 3, 2, 5, 2, 4, 2, 1, 2, 2, 4, 2, 3, 1, 2, 4, 4, 5, 3, 2, 4, 3, 1, 3, 4, 1, 1, 1, 4, 4, 2, 5, 3, 5, 3, 4, 3, 3, 2, 4, 3, 4, 4, 1, 2, 4, 3, 4, 3, 5, 1, 1, 4, 5, 4, 5, 3, 1, 4, 2, 2, 2, 2, 5, 2, 5, 4, 5, 5, 1, 3, 1, 3, 4, 3, 2, 5, 5, 3, 1, 5, 2, 3, 3, 4, 5, 1, 1, 2, 5, 5, 4, 5, 3, 1, 4, 4, 3, 3, 1, 2, 2, 4, 3, 1, 3, 2, 5, 1, 1, 5, 4, 2, 4, 3, 4, 3, 1, 3, 4, 5, 2, 4, 3, 3, 1, 1, 3, 4, 3, 3, 2, 5, 5, 5, 2, 4, 1, 4, 4, 4, 3, 1, 4, 3, 2, 5, 1, 2, 1, 2, 5, 4, 4, 2, 3, 4, 4, 3, 5, 2, 4, 5, 3, 5, 1, 1, 5, 4, 3, 2, 4, 5, 5, 5, 4, 2, 1, 1, 2, 3, 5, 5, 3, 3, 5, 1, 3, 5, 3, 3, 1, 1, 5, 5, 2, 5, 1, 2, 1, 1, 3, 3, 4, 1, 5, 2, 1, 1, 2, 3, 1, 3, 3, 2, 5, 2, 4, 5, 2, 1, 3, 4, 3, 4, 2, 5, 4, 3, 5, 5, 5, 5, 2, 3, 5, 4, 1, 3, 4, 1, 2, 1, 5, 3, 2, 5, 1, 4, 1, 4, 1, 1, 3, 4, 4, 2, 1, 1, 1, 3, 4, 4, 4, 5, 2, 1, 1, 1, 3, 2, 2, 2, 5, 2, 1, 5, 3, 4, 4, 3, 2, 5, 3, 1, 5, 4, 1, 1, 2, 5, 1, 3, 4, 4, 3, 4, 1, 2, 3,

In [ ]:
i = 3

min_leng = min(len(column1[i]),len(column2[i]))
column1[i] = column1[i][:min_leng]
column2[i] = column2[i][:min_leng]

type(column2[i][0])

str

In [ ]:
# Calculate F1 score and mAP for each pair of scores
f1_scores = []
mAPs = []

for i in range(len(total_last_df)):
    column1_values = column1[i]
    column2_values = column2[i]

    min_length = min(len(column1_values), len(column2_values))
    column1_values = column1_values[:min_length]
    column2_values = column2_values[:min_length]

    f1 = f1_score(column1_values, column2_values, average='weighted')
    mAP = average_precision_score(column1_values, column2_values)

    f1_scores.append(f1)
    mAPs.append(mAP)

# Assign F1 score and mAP to DataFrame
total_last_df['F1_Score'] = f1_scores
total_last_df['mAP'] = mAPs

# Print DataFrame with video_id, F1_Score, and mAP columns
print(total_last_df[['video_id', 'F1_Score', 'mAP']])

In [ ]:
# Save DataFrame to CSV
total_last_df[['video_id', 'F1_Score', 'mAP']].to_csv('metrics_scores.csv', index=False)